In [2]:
cd ../..

/home/s377963/car_t_sc


In [3]:
#import required packages
import scanpy as sc
import scanpy.external as sce
import h5py
# import rpy2
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import sys
sys.path.append('/home/s377963/car_t_sc/02_code/functions')
import functions_dat_processing as dp
%load_ext rpy2.ipython

In [4]:
#import data from cellranger
adata = sc.read_10x_mtx("./01_data/raw/cellranger_multi/2024-06-07_24054SC_Luu_P1_cellranger/per_sample_outs/count/sample_filtered_feature_bc_matrix", gex_only=False)

# Quality Control

1. Filter for low quality cells / likely dying cells based on mitochondrial counts per cell; total gene count per cell and total counts per cell

In [8]:
# select the features of interest for quality control, in this case HTOs and mitochondiral features:
#mitochondrial:
adata.var['mt'] = adata.var_names.str.startswith('mt-') #note for mice its mt for human its MT
#HTOs
adata.var['hto'] = adata.var['feature_types'] == 'Antibody Capture'

In [9]:
adata.var

,gene_ids,feature_types,mt,hto
Xkr4,ENSMUSG00000051951,Gene Expression,False,False
Gm1992,ENSMUSG00000089699,Gene Expression,False,False
Gm19938,ENSMUSG00000102331,Gene Expression,False,False
Gm37381,ENSMUSG00000102343,Gene Expression,False,False
Rp1,ENSMUSG00000025900,Gene Expression,False,False
...,...,...,...,...
ENSMUSG00000095019,ENSMUSG00000095019,Gene Expression,False,False
ENSMUSG00000095041,ENSMUSG00000095041,Gene Expression,False,False
Ms.Hashtag_1,d0_C,Antibody Capture,False,True
Ms.Hashtag_2,d0_P,Antibody Capture,False,True
